In [1]:
import json
import os
import pandas as pd
import glob

# EDA

In [ ]:
def open_json(file_name, direction):
    main_direction = os.getcwd()
    full_direction = os.path.join(main_direction, direction)
    file_location = os.path.join(full_direction, file_name)
    with open(file_location, 'r') as file:
        data = json.load(file)
    print(f'data named "\{file_name}\" in direction "\{direction}\" is loaded!')
    return data

def print_data(name:str, data):
    print(name)
    print('type of data {}: '.format(type(data)))
    main_keys = list(data.keys())
    value_type = [type(data[key]) for key in main_keys]
    tuple_data = [(main_keys[i], value_type[i]) for i in range(len(main_keys))]
    print('\n(key, type):')
    for element in tuple_data:
        print(element)
    print('-' * 10)
    return None

In [5]:
direction = r'data/data/raw_data/2016/playoff_games'
file_name = '2016030111.json'
data = open_json(file_name, direction)
print('length of data:', len(data))

#print_data('data', data)
#print_data('gameData', data['gameData'])
#print_data('liveData', data['liveData'])

print('\nlength of gameData: {}'.format(len(data['gameData'])))
print('length of liveData: {}'.format(len(data['liveData'])))

player_ids = data['gameData']['players'].keys()
print('player ids', player_ids, '\n')

player_info_example = data['gameData']['players']['ID8471228'] # player, team
print("example of info. for a player ")
print(player_info_example)
print('\n')

# the data about events in the game
events = data['liveData']['plays']['allPlays']
print('length of events: '.format(len(events)))
print('example of an event: \n', events[10])

data named "\2016030111.json" in direction "\data/data/raw_data/2016/playoff_games" is loaded!
length of data: 6

length of gameData: 6
length of liveData: 4
player ids dict_keys(['ID8471958', 'ID8467496', 'ID8476808', 'ID8475715', 'ID8471679', 'ID8474151', 'ID8476851', 'ID8475763', 'ID8474157', 'ID8469521', 'ID8473507', 'ID8471686', 'ID8470630', 'ID8474038', 'ID8475848', 'ID8475204', 'ID8476459', 'ID8473546', 'ID8477476', 'ID8476300', 'ID8471296', 'ID8478561', 'ID8475692', 'ID8470041', 'ID8474000', 'ID8476381', 'ID8468685', 'ID8476869', 'ID8476468', 'ID8471338', 'ID8474207', 'ID8470642', 'ID8475855', 'ID8476431', 'ID8477402', 'ID8476477', 'ID8477446', 'ID8475184', 'ID8474090', 'ID8476470', 'ID8476918', 'ID8474818', 'ID8474613', 'ID8475869', 'ID8476479', 'ID8470854', 'ID8471228', 'ID8474100', 'ID8468505']) 

example of info. for a player 
{'id': 8471228, 'fullName': 'Alexander Radulov', 'link': '/api/v1/people/8471228', 'firstName': 'Alexander', 'lastName': 'Radulov', 'primaryNumber': 

In [4]:
data.keys()

dict_keys(['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

In [9]:
data['gameData']['players'].keys()

dict_keys(['ID8471958', 'ID8467496', 'ID8476808', 'ID8475715', 'ID8471679', 'ID8474151', 'ID8476851', 'ID8475763', 'ID8474157', 'ID8469521', 'ID8473507', 'ID8471686', 'ID8470630', 'ID8474038', 'ID8475848', 'ID8475204', 'ID8476459', 'ID8473546', 'ID8477476', 'ID8476300', 'ID8471296', 'ID8478561', 'ID8475692', 'ID8470041', 'ID8474000', 'ID8476381', 'ID8468685', 'ID8476869', 'ID8476468', 'ID8471338', 'ID8474207', 'ID8470642', 'ID8475855', 'ID8476431', 'ID8477402', 'ID8476477', 'ID8477446', 'ID8475184', 'ID8474090', 'ID8476470', 'ID8476918', 'ID8474818', 'ID8474613', 'ID8475869', 'ID8476479', 'ID8470854', 'ID8471228', 'ID8474100', 'ID8468505'])

In [15]:
data['liveData'].keys()

dict_keys(['plays', 'linescore', 'boxscore', 'decisions'])

In [17]:
data['liveData']['plays']['scoringPlays']

[64, 383]

# Importing JSON and building dataframes

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
spark = (
    SparkSession.builder
    .config('spark.executor.memory', '5g')
    .config('spark.executor.cores', '4')
    .config('spark.driver.memory','3g')
    .config('spark.cores.max', '300')
    .config('spark.sql.debug.maxToStringFields','50')
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/07 12:55:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
# Read JSON files into a DataFrame
df = spark.read.json("../data/data/raw_data/2016/regular_games/2016020001.json")
schema = df.schema
data_2016 = spark.read.option("recursiveFileLookup", "true").json("../data/data/raw_data/*/*/*.json", schema=schema)

In [50]:
data_2016.select(F.countDistinct("gameData.game.pk")).show()

+--------------------------------+
|count(DISTINCT gameData.game.pk)|
+--------------------------------+
|                            7461|
+--------------------------------+



In [51]:
data_2016.count()

177866

In [9]:
#data_2016.printSchema()

In [5]:
gameData_season_summary = data_2016.select(F.col("gameData.game.pk").alias('game_id'),
                                        F.col("gameData.game.season").alias("season"),
                                        F.col("gameData.game.type").alias("game_type"),
                                        F.col("gameData.datetime.dateTime").alias("start_time"),
                                        F.col("gameData.datetime.endDateTime").alias("end_time"),
                                        F.col("gameData.teams.away.id").alias("away_team_id"),
                                        F.col("gameData.teams.away.name").alias("away_team_name"),
                                        F.col("gameData.teams.home.id").alias("home_team_id"),
                                        F.col("gameData.teams.home.name").alias("home_team_name")
                                        )

In [6]:
gameData_season_summary.select(F.countDistinct("game_id")).show()

+-----------------------+
|count(DISTINCT game_id)|
+-----------------------+
|                   1268|
+-----------------------+



In [11]:
allplay = data_2016.select(F.col("gamePk").alias('game_id'),
                           F.col("gameData.game.season").alias("season"),
                            F.col("gameData.game.type").alias("game_type"),
                            F.col("gameData.datetime.dateTime").alias("start_time"),
                            F.col("gameData.datetime.endDateTime").alias("end_time"),
                            F.col("gameData.teams.away.id").alias("away_team_id"),
                            F.col("gameData.teams.away.name").alias("away_team_name"),
                            F.col("gameData.teams.home.id").alias("home_team_id"),
                            F.col("gameData.teams.home.name").alias("home_team_name"),
                            F.explode(F.col("liveData.plays.allplays")).alias("allplays")
                                        )

linescore = data_2016.select(F.col("gamePk").alias('game_id'),
                             F.explode(F.col("liveData.linescore.periods")).alias("periods")
                                        )

In [10]:
#allplay.printSchema()

In [6]:
allplay.select(F.countDistinct("game_id")).show()

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=35604Kb max_used=35767Kb free=95467Kb
 bounds [0x00000001089d8000, 0x000000010ad18000, 0x00000001109d8000]
 total_blobs=13352 nmethods=12436 adapters=827
 compilation: disabled (not enough contiguous free space left)


+-----------------------+
|count(DISTINCT game_id)|
+-----------------------+
|                   1264|
+-----------------------+



In [12]:
allplay = allplay.filter((allplay.allplays.result.event=="Goal") | (allplay.allplays.result.event=="Shot"))

In [9]:
allplay.select(F.countDistinct("game_id")).show()

+-----------------------+
|count(DISTINCT game_id)|
+-----------------------+
|                   1264|
+-----------------------+



In [17]:
allplayDF = allplay.select(F.col("game_id"),
                           F.col("season").alias("season"),
                            F.col("game_type").alias("game_type"),
                            F.col("start_time").alias("start_time"),
                            F.col("end_time").alias("end_time"),
                            F.col("away_team_id").alias("away_team_id"),
                            F.col("away_team_name").alias("away_team_name"),
                            F.col("home_team_id").alias("home_team_id"),
                            F.col("home_team_name").alias("home_team_name"),
                            F.col("allplays.about.eventIdx").alias("eventIdx"),
                           F.col("allplays.result.event").alias("event"),
                           F.col("allplays.result.description").alias("description"),
                           F.col("allplays.result.secondaryType").alias("shotType"),
                           F.col("allplays.result.strength.code").alias("strength"),
                           F.col("allplays.result.gameWinningGoal").alias("gameWinningGoal"),
                           F.col("allplays.result.emptyNet").alias("emptyNet"),
                            F.col("allplays.result.penaltySeverity").alias("penaltySeverity"),
                            F.col("allplays.result.penaltyMinutes").alias("penaltyMinutes"),
                            F.col("allplays.coordinates.x").alias("x_coordinate"),
                            F.col("allplays.coordinates.y").alias("y_coordinate"),
                            F.col("allplays.team.name").alias("team_name"),
                            F.col("allplays.about.period").alias("period"),
                            F.col("allplays.about.periodType").alias("periodType"),
                            F.col("allplays.about.periodTime").alias("periodTime"),
                            F.col("allplays.about.periodTimeRemaining").alias("periodTimeRemaining"),
                            F.when(F.col("allplays.result.event")=="Goal",1).otherwise(0).alias("is_goal"),
                            F.explode(F.col("allplays.players")).alias("players")
                            )

In [14]:
#allplayDF.printSchema()

In [18]:
player_characteristics = allplayDF.groupBy(["game_id","eventIdx"]).pivot("players.playerType").agg(F.first("players.player.fullName"),F.last("players.player.fullName"))

player_characteristics = player_characteristics.withColumnRenamed("Assist_first(players.player.fullName)","Assist_first")\
        .withColumnRenamed("Assist_last(players.player.fullName)","Assist_last")\
            .withColumnRenamed("Scorer_first(players.player.fullName)","Scorer")\
                .withColumnRenamed("Goalie_first(players.player.fullName)","Goalie")\
                    .withColumnRenamed("Shooter_first(players.player.fullName)","Shooter")

player_characteristics = player_characteristics.select("game_id","eventIdx","Assist_first","Assist_last","Scorer","Goalie","Shooter")

In [19]:
allplayDF = allplayDF.join(player_characteristics,["game_id","eventIdx"],"left").drop("players").drop_duplicates()

In [20]:
linescore.printSchema()

root
 |-- game_id: long (nullable = true)
 |-- periods: struct (nullable = true)
 |    |-- away: struct (nullable = true)
 |    |    |-- goals: long (nullable = true)
 |    |    |-- rinkSide: string (nullable = true)
 |    |    |-- shotsOnGoal: long (nullable = true)
 |    |-- endTime: string (nullable = true)
 |    |-- home: struct (nullable = true)
 |    |    |-- goals: long (nullable = true)
 |    |    |-- rinkSide: string (nullable = true)
 |    |    |-- shotsOnGoal: long (nullable = true)
 |    |-- num: long (nullable = true)
 |    |-- ordinalNum: string (nullable = true)
 |    |-- periodType: string (nullable = true)
 |    |-- startTime: string (nullable = true)



In [21]:
linescore = linescore.select("game_id",
                            F.col("periods.num").alias("period"),
                            F.col("periods.home.rinkSide").alias("home_rinkSide"),
                            F.col("periods.away.rinkSide").alias("away_rinkSide"),
            )

allplayDF = allplayDF.join(linescore,["game_id","period"],"left").drop_duplicates()

In [63]:
allplayDF.write.option("compression", "snappy").mode("overwrite").parquet('data/playData.parquet')

In [22]:
allplayDF = allplayDF.toPandas()

In [23]:
allplayDF.shape

(454011, 33)

In [25]:
allplayDF.to_csv('../data/playData_v2.csv',index=False)

23/10/08 10:58:13 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 166088 ms exceeds timeout 120000 ms
23/10/08 10:58:13 WARN SparkContext: Killing executors is not supported by current scheduler.
23/10/08 10:58:20 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [73]:
allplayDF.repartition(1).write.mode('overwrite').csv('data/playData2_sw.csv')

In [67]:
allplayDF.shape

(454011, 31)